In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(3)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [ ]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/WganModelV2_16_MultiGood2')


dpath = DATA_PATH/(proj_id + '_54332_D.h5')
gpath = DATA_PATH/(proj_id + '_54332_G.h5')
keep_pct=1.0
bs=8
sz=224

c_lr=6e-4
c_lrs = np.array([c_lr,c_lr,c_lr,c_lr])

g_lr=2e-4
g_lrs = np.array([g_lr/10000,g_lr/100,g_lr])

torch.backends.cudnn.benchmark=True

## Training

In [ ]:
netG = Unet34(nf_factor=2).cuda()
netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=128, sz=sz, scale=16).cuda()
netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [ ]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [ ]:
trainer = WGANTrainer(netD=netD, netG=netG, md=md, bs=bs, sz=sz, dpath=dpath, gpath=gpath, genloss_fns=[FeatureLoss(multiplier=1e5)])
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer')

In [ ]:
#netG.freeze_to(1)
trainer.train(lrs_critic=c_lrs, lrs_gen=g_lrs, clr_critic=(1,8, 0.9, 0.9), clr_gen=(1,8, 0.9, 0.9), cycle_len=1, epochs=1)


WDist -98.3441390991211; RScore -108.22945404052734; FScore -206.57359313964844; GAddlLoss 1665.4921875; GCount: 48400; GPenalty: 11.772364616394043; GCost: 151.54843139648438; ConPenalty: 52.19490432739258
 61%|██████    | 96820/158545 [15:22:50<8:46:47,  1.95it/s]
WDist -156.45787048339844; RScore 264.54266357421875; FScore 108.08479309082031; GAddlLoss 2258.3359375; GCount: 48410; GPenalty: 29.122684478759766; GCost: -44.19493103027344; ConPenalty: 75.19851684570312
 61%|██████    | 96840/158545 [15:23:00<8:53:07,  1.93it/s]
WDist -144.06027221679688; RScore 221.76754760742188; FScore 77.70726776123047; GAddlLoss 1917.5963134765625; GCount: 48420; GPenalty: 27.05652618408203; GCost: -92.35336303710938; ConPenalty: 167.45034790039062
 61%|██████    | 96860/158545 [15:23:10<8:42:31,  1.97it/s]
WDist -108.52520751953125; RScore 141.55657958984375; FScore 33.031375885009766; GAddlLoss 2207.07470703125; GCount: 48430; GPenalty: 20.82954216003418; GCost: 89.9416732788086; ConPenalty: 81.